In [77]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from scipy.special import softmax
from sklearn.metrics import accuracy_score as ac

In [78]:
data=load_digits()

In [79]:
data

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [98]:
#normalizamos los datos
X=MinMaxScaler().fit_transform(data.data)
X

array([[0.    , 0.    , 0.3125, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.625 , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 1.    , 0.5625, 0.    ],
       ...,
       [0.    , 0.    , 0.0625, ..., 0.375 , 0.    , 0.    ],
       [0.    , 0.    , 0.125 , ..., 0.75  , 0.    , 0.    ],
       [0.    , 0.    , 0.625 , ..., 0.75  , 0.0625, 0.    ]])

In [81]:
y=LabelBinarizer().fit_transform(data.target)
y

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [82]:
y.shape[1]

10

In [83]:
skf=StratifiedKFold(n_splits=10,shuffle=True,random_state=0)

In [97]:
df=pd.DataFrame(columns=['k','Acc.SVM-Poly2','Acc.SVM-Poly3','Acc.SVM-Poly4','Acc.SVM-Poly5','Acc.SVM-Poly6','Acc.SVM-Poly7','Acc.SVM-Poly8','Acc.SVM-Poly9','Acc.SVM-Poly10'])
k=0
for train_index, test_index in skf.split(X, y[:,0]):
    X_train, X_test=X[train_index], X[test_index]
    y_train,y_test=y[train_index], y[test_index] 
    y_predict =np.ones(y_test.shape, dtype=float)
    y_predict_train =np.ones(y_train.shape, dtype=float)
    #rlm para cada una de las clases
    for i in range(y.shape[1]):
        
        clf=SGDClassifier().fit(X_train,y_train[:,i])
        
        y_predict[:,i]=clf.predict(X_test)
        y_predict_train[:,i]=clf.predict(X_train)
    #clasificador predicho con mayor tasa de acierto para asignarle esa clase
    y_predict_test = np.argmax(y_predict, axis=1)  
    
    #Acierto de rlm con cada carpeta
    print('Acierto rlm carpeta k',k,'acierto',(ac(np.argmax(y_test,axis=1),np.argmax(y_predict, axis=1))))
                
    
    m1=softmax(X_train,axis=1)
    m2=softmax(y_predict_train,axis=1)
    m3=softmax(X_test,axis=1)
    m4=softmax(y_predict,axis=1)
    min_degree=2
    max_degree=10
    
    for degree in range (min_degree,max_degree+1):
        svm = SVC(kernel='poly', degree=degree, random_state=0, gamma='auto')
        svm.fit(m1,y_predict_train[:,i])
    
        accuracy = svm.score(m3,y_predict[:,i])
        #print(accuracy)
    #print()
    k+=1

Acierto rlm carpeta k 0 acierto 0.8833333333333333
Acierto rlm carpeta k 1 acierto 0.8777777777777778
Acierto rlm carpeta k 2 acierto 0.9
Acierto rlm carpeta k 3 acierto 0.9611111111111111
Acierto rlm carpeta k 4 acierto 0.9222222222222223
Acierto rlm carpeta k 5 acierto 0.9222222222222223
Acierto rlm carpeta k 6 acierto 0.9
Acierto rlm carpeta k 7 acierto 0.8938547486033519
Acierto rlm carpeta k 8 acierto 0.9162011173184358
Acierto rlm carpeta k 9 acierto 0.8547486033519553
